In [5]:
import numpy as np
import os
import os.path
from astroquery.jplhorizons import Horizons

In [6]:
# Paths to directories containing data
home_path = os.path.expanduser('~/')
path_67p = home_path+'Documents/year1/shape_modelling/67p/'

In [7]:
# Constants
km2au = 1.496e8
c = 3e5   #km/s
comet_id = '90000696'

In [8]:
### Function to create a file in the 'Mikko' format required for shape modelling 

def mikkoWrite(filename,day_list,int_list,sunx,suny,sunz,earthx,earthy,earthz, min_dist):
# Code to write to lightcurve file in Mikko format
# Additional argument to limit files to points beyond min_dist from the sun

    counter = 0
    fout = open(filename, 'w+')
    sameday = []
    for i, day in enumerate(day_list[:-1]):
        if (np.sqrt(sunx[i]**2+suny[i]**2+sunz[i]**2) >= min_dist):
            newday = True
            sameday.append(day)
            if abs(day-day_list[i+1])<0.5:
                newday = False
            if newday==False:
                if day not in sameday:
                    sameday.append(day)
            else:
                fout.write('%i 1\n'%(len(sameday)))
                for time in sameday:
                    loc = day_list.index(time)
                    fout.write('%.6f %.8f %.10f %.8f %.8f %.8f %.8f %.8f\n'%(time+2400000.5,int_list[loc],sunx[loc], suny[loc], sunz[loc], earthx[loc], earthy[loc], earthz[loc]))
                counter = counter+1
                sameday = []
    fout.write('1 1\n')
    fout.write('%.6f %.8f %.10f %.8f %.8f %.8f %.8f %.8f\n'%(day_list[-1]+2400000.5,int_list[-1],sunx[-1], suny[-1], sunz[-1], earthx[-1], earthy[-1], earthz[-1]))
    counter = counter+1
    fout.close()
    return counter

In [9]:
# Read LSST sim output

lsst_file_name = path_67p+'LSST_out_LC_67P.txt'
lsst_file = open(lsst_file_name, 'r')

lsst_mjd =[]; lsst_delta=[]
lsst_ra = []; lsst_dec = []
lsst_sunx = []; lsst_suny = []; lsst_sunz = []
lsst_phase_ang = [];
lines_lsst = lsst_file.readlines()
for line in lines_lsst[25:]:
    dat = line.split()
    lsst_mjd.append(float(dat[2]))
    lsst_delta.append(float(dat[3]))
    lsst_ra.append(float(dat[5]))
    lsst_dec.append(float(dat[7]))
    lsst_sunx.append(float(dat[9]))
    lsst_suny.append(float(dat[10]))
    lsst_sunz.append(float(dat[11]))
    lsst_phase_ang.append(float(dat[21]))

lsst_delta = np.array(lsst_delta)/km2au
lsst_sunx = np.array(lsst_sunx)/km2au
lsst_suny = np.array(lsst_suny)/km2au
lsst_sunz = np.array(lsst_sunz)/km2au

In [10]:
# Check it got everything
print(len(lsst_mjd))

678


In [11]:
mjd = np.array(lsst_mjd)
sun_x = lsst_sunx
sun_y = lsst_suny
sun_z = lsst_sunz

- - - - - - - - - - - - -

In [12]:
# LSST output only has obj-Sun geom: also need Earth-obj
# Also want list of solar phase angles, to compute abs mag from phase function
# ^Phase angles in LSST output

# Splitting this into 2 because Horizons wouldn't let me do all 678 epochs at once :(
obj = Horizons(id=comet_id, location='I11',epochs=mjd[:500])

In [16]:
earth_x = []; earth_y=[]; earth_z=[];
vec = obj.vectors(refplane='earth')  # LSST vectoral output in equatorial ref frame!!! Horizons defaults to ecliptic

for i in range(len(vec)):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [17]:
# Check output looks sensible
vec

targetname,datetime_jd,datetime_str,M1,x,k1,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,AU,---,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str25,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
67P/Churyumov-Gerasimenko,2459953.863937,A.D. 2023-Jan-09 08:44:04.1568,13.9,-3.444445289378263,7.75,-2.015330048691382,-0.78247324275391,0.01366221321528024,-0.001412607087569043,-0.001023160365304245,0.02348728196591665,4.066696808504949,-0.01067482600168162
67P/Churyumov-Gerasimenko,2459956.868077,A.D. 2023-Jan-12 08:50:01.8528,13.9,-3.403917467122777,7.75,-2.019038534981401,-0.7850220821185131,0.01338745065129749,-0.0006023355223158401,-0.0006748480944670384,0.02330293207557787,4.034777614458704,-0.010861532409711
67P/Churyumov-Gerasimenko,2459957.822871,A.D. 2023-Jan-13 07:44:56.0544,13.9,-3.391208755252286,7.75,-2.019722018595642,-0.7856140386169299,0.01335622468733235,-0.0003645603337652831,-0.0005655073426604261,0.02324369050000328,4.024520253616798,-0.01096109903201675
67P/Churyumov-Gerasimenko,2459964.842786,A.D. 2023-Jan-20 08:13:36.7104,13.9,-3.300777863744525,7.75,-2.017353466688469,-0.7868077298820525,0.01248799001219692,0.001465100802278048,0.0002184380828881965,0.02279970694340666,3.947646883796473,-0.01123392675560529
67P/Churyumov-Gerasimenko,2459966.871729,A.D. 2023-Jan-22 08:55:17.3856,13.9,-3.275792893852368,7.75,-2.014326365155931,-0.786142529920415,0.01215487368252741,0.001970705116553976,0.0004364122010969505,0.02266944275386499,3.925092338547674,-0.01124293914504588
67P/Churyumov-Gerasimenko,2459966.876288,A.D. 2023-Jan-22 09:01:51.2832,13.9,-3.275737496452676,7.75,-2.014317379356756,-0.7861405391803625,0.01214756356086103,0.001971267497432691,0.0004369109092912314,0.02266914679607258,3.925041095044421,-0.01123719730215952
67P/Churyumov-Gerasimenko,2459970.830448,A.D. 2023-Jan-26 07:55:50.7072,13.9,-3.228795234056606,7.75,-2.005559087153143,-0.7835972745480004,0.01158921170090879,0.002916277921828557,0.000845763806308691,0.02241422828581285,3.880903323223635,-0.01131970929134069
67P/Churyumov-Gerasimenko,2459970.830859,A.D. 2023-Jan-26 07:56:26.2176,13.9,-3.228790471027133,7.75,-2.005557888528299,-0.7835969269303094,0.01158854738402633,0.002916446149405821,0.0008458064746746845,0.02241420141633283,3.880898670917385,-0.01131925052089757


In [18]:
# Now get the remaining 655-500 vectors
obj = Horizons(id=comet_id, location='I11', epochs=mjd[500:])

In [19]:
vec = obj.vectors(refplane='earth')  
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [21]:
#for i in range(678):
#    print(f'{ecl_lon[i]:.4f}  {ecl_lat[i]:.4f}')

In [22]:
# Check this didn't miss anything or double anything
len(earth_x)

678

In [23]:
# Light time correct the MJDs:
delta = lsst_delta # in km
#for i in range(len(earth_x)):
#    delta.append(np.sqrt(earth_x[i]**2+earth_y[i]**2+earth_z[i]**2))
#delta=np.array(delta)
mjd_ltc = np.zeros_like(mjd)

mjd_ltc = mjd-(((delta*km2au)/c)/(3600*24))
mjd_ltc = np.ndarray.tolist(mjd_ltc)

In [24]:
# Just checking lighttime corrected MJDs look right:
for i,time in enumerate(mjd_ltc):
    print(time, mjd[i])

59953.34046686406 59953.363937
59956.34479107718 59956.368077
59957.29964427473 59957.322871
59964.320002923756 59964.342786
59966.3490760892 59966.371729
59966.353635384934 59966.376288
59970.30805011133 59970.330448
59970.30846113815 59970.330859
59970.323103093804 59970.3455
59970.323931147774 59970.346328
59972.297099372634 59972.319369
59973.35521357923 59973.377415
59977.31767781625 59977.339625
59978.30851600644 59978.3304
59979.344438877495 59979.366257
59980.278004965185 59980.299764
59984.24211719965 59984.263628
59984.245833433575 59984.267344
59984.25742616275 59984.278936
59984.25825021452 59984.27976
59984.30237198136 59984.323879
59990.31762575978 59990.338768
59990.333675713955 59990.354817
59990.349401646774 59990.370542
59991.25495492466 59991.276042
59991.271517907946 59991.292604
59992.31625583932 59992.337281
59992.33249678522 59992.353521
59994.31207227407 59994.332983
59994.31248729774 59994.333398
59994.32763415996 59994.348544
59994.32805218372 59994.348962
599

In [25]:
sun_xyz = np.array([sun_x, sun_y, sun_z])
print(sun_xyz[:,0])
earth_xyz = np.array([earth_x, earth_y, earth_z])
print(earth_xyz[:,0])

[-3.75625676 -1.15950396 -0.41150245]
[-3.44444529 -2.01533005 -0.78247324]


In [67]:
# Creating Mikko file with dummy intensities. 
namestr = path_67p+'test.txt'
#"""
if (os.path.exists(namestr)==True):
    print(f'Just checking you definitely want to overwrite {namestr[-8:]}?')

else:
#"""
    mikkoWrite(namestr,mjd_ltc,np.ones_like(mjd),sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,0.0)
    
# ~~ ----- ACTION REQUIRED ---- ~~ #
##### Go and type the number output from this cell into the top of the file you've just created #####
# This is the number of lightcurves in total

Just checking you definitely want to overwrite test.txt?


In [30]:
# Checking the phase angles: should make this into a function but too lazy
alpha_list_calc = []
for i in range(len(earth_xyz[1])):
    alpha_list_calc.append((360/(2.*np.pi))*np.arccos(np.dot(sun_xyz[:,i],earth_xyz[:,i])/(np.linalg.norm(sun_xyz[:,i])*np.linalg.norm(earth_xyz[:,i]))))

In [36]:
# This is just to grab ephemeris data to check phase angles
# Will also use this to grab rh values for completion's sake
obj = Horizons(id=comet_id, location='I11', epochs=mjd[:500])
ephem_table = obj.ephemerides()

In [37]:
ephem_table

targetname,datetime_str,datetime_jd,M1,solar_presence,k1,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,Tmag,Nmag,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
---,---,d,mag,---,---,---,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,---,---,mag,mag,mag,%,arcsec,arcsec,---,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,---,deg,deg,%,deg,deg,deg,deg,---,s,deg,deg,deg,deg,deg,deg,---,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,---,deg,deg,deg
str25,str24,float64,float64,str1,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,str1,float64,int64,int64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
67P/Churyumov-Gerasimenko,2023-Jan-09 08:44:04.157,2459953.863937002,13.9,A,7.75,m,210.33036,-11.09283,210.63394,-11.20291,17.16243,-6.60705,72.894647,46.918734,-477.69,744.99,-265912.02,39290.171,263.255,11.2608711845,1.367,0.19,21.572,--,98.51569,2e-05,275038.5,*,0.001356,--,--,--,--,110.3,0.0,--,--,198.0968,1.2131,3.952678408776,8.8807921,4.06653642809274,-18.4812816,33.82035219,13.8660163,23.844665,76.3996,/L,13.9944,78.1,94.9524,89.608,290.366,295.164,-0.89209,Vir,69.184123,212.4111735,1.1782282,--,--,329.156004,48.189232,3.9031929991,0.000354,0.259,0.164,0.302,0.057,-31.372,0.0543561,0.307,148.2772,1.38e-05,0.13,0.46,0.000989,139.5795,-2.781391273,13.9957,205.0942,1.2057
67P/Churyumov-Gerasimenko,2023-Jan-12 08:50:01.853,2459956.868077,13.9,A,7.75,m,210.67301,-11.21862,210.9772,-11.32847,15.65553,-6.03624,69.69044,50.349695,-498.15,731.03,-276526.81,37197.461,262.024,11.5576319129,1.297,0.18,21.568,--,98.49606,2.1e-05,284760.0,*,0.001367,--,--,--,--,110.17,0.0,--,--,198.3631,1.196,3.968045653476,8.8321869,4.03461887013207,-18.8044217,33.55490195,13.8032156,23.2297669,79.1,/L,14.0874,42.3,75.9931,86.8147,290.233,295.123,-0.91331,Vir,69.184209,212.7705334,1.1752859,--,--,329.54056,47.938291,3.9825239613,0.000354,0.262,0.166,0.305,0.058,-31.312,0.0553718,0.311,149.7997,1.38e-05,0.1,0.36,0.000999,139.8462,-2.507514424,14.0884,205.4069,1.1957
67P/Churyumov-Gerasimenko,2023-Jan-13 07:44:56.054,2459957.822871,13.9,,7.75,m,210.77565,-11.25618,211.08001,-11.36595,15.26725,-5.87238,80.823989,37.474016,-436.14,769.5,-279940.16,36492.452,261.64,10.5354261699,1.64,0.228,21.567,--,98.49111,2.1e-05,287872.1,*,0.00137,--,--,--,--,110.13,0.0,--,--,198.4473,1.1905,3.972912124229,8.8167776,4.02436201041572,-18.9771862,33.46959825,13.78337,23.1522986,79.9645,/L,14.1107,30.4,67.3944,85.9268,290.192,295.111,-0.91968,Vir,69.184236,212.8780974,1.1744169,--,--,329.654973,47.862937,2.8915459744,0.000354,0.264,0.167,0.306,0.058,-31.294,0.0557046,0.312,150.2082,1.38e-05,0.09,0.33,0.001002,139.9305,-3.536574242,14.1117,205.5027,1.1925
67P/Churyumov-Gerasimenko,2023-Jan-20 08:13:36.710,2459964.842786,13.9,,7.75,m,211.43087,-11.49602,211.73676,-11.6054,11.45893,-4.47512,71.727685,48.869

In [39]:
alpha_list = []; rh_list = []; ecl_lon = []; ecl_lat = []

for i,row in enumerate(ephem_table['alpha_true']):
    alpha_list.append(float(row))
    rh_list.append(float(ephem_table['r'][i]))
    ecl_lon.append(float(ephem_table['EclLon'][i]))
    ecl_lat.append(float(ephem_table['EclLat'][i]))

In [40]:
obj = Horizons(id=comet_id, location='I11', epochs=mjd[500:])
ephem_table = obj.ephemerides()

In [41]:
# Append the rest of the values to the lists
for i,row in enumerate(ephem_table['alpha_true']):
    alpha_list.append(float(row))
    rh_list.append(float(ephem_table['r'][i]))
    ecl_lon.append(float(ephem_table['EclLon'][i]))
    ecl_lat.append(float(ephem_table['EclLat'][i]))

In [42]:
for i in range(len(alpha_list)):
    print(alpha_list[i], lsst_phase_ang[i])

13.9957 13.993685
14.0884 14.086378
14.1117 14.109652
14.1847 14.182674
14.1725 14.170457
14.1724 14.170399
14.1039 14.101938
14.1039 14.101926
14.1035 14.101524
14.1035 14.101501
14.0466 14.044583
14.0095 14.007557
13.8318 13.829852
13.7775 13.775489
13.7162 13.714225
13.6575 13.65551
13.3671 13.365123
13.3668 13.364812
13.3658 13.363841
13.3657 13.363772
13.362 13.360051
12.7926 12.790664
12.7908 12.788885
12.7891 12.787139
12.6901 12.688142
12.6882 12.686249
12.569 12.567086
12.5671 12.565145
12.3285 12.326588
12.3285 12.326535
12.3265 12.324593
12.3265 12.324539
11.9404 11.938493
11.9404 11.938435
11.938 11.936064
11.9379 11.936005
11.1933 11.19135
11.1906 11.188692
11.0331 11.031215
11.0304 11.028539
10.5441 10.542192
10.5389 10.537024
10.1746 10.172745
9.9955 9.993589
8.3859 8.384018
8.3822 8.380298
8.3788 8.376931
7.9551 7.953266
7.9515 7.949652
6.5691 6.567276
6.5633 6.561435
6.5586 6.556755
5.5922 5.59033
5.592 5.590119
5.588 5.586163
5.5878 5.585951
5.5788 5.576914
5.5747 5.5

In [ ]:
####-----Go and use mag_from_phase_func.py to get shifted absolute magnitudes-----#####

In [43]:
# Read in shifted magnitudes:
shifted_mags_file = path_67p+'67P_mag_list_ALL.txt'
shifted_mags_list = []
magf=open(shifted_mags_file, 'r')
lines = magf.readlines()
for line in lines:
    data = line.split()
    shifted_mags_list.append(float(data[0]))
#shifted_mags_list
magf.close()

In [45]:
corr_fac = 16    # from mag_from_phase_func py script

generate_intensities = []
for i in range(len(shifted_mags_list)):
    intensity=10.**(-0.4*(shifted_mags_list[i]-corr_fac))
    generate_intensities.append(intensity)

In [46]:
# Final step: read in intensities as calculated from shifts from Matlab script
# Obvs if you're using this script to create Mikko file in the first instance, you won't have these
intensities_file = path_67p+'67P_int_list_ALL.txt'
intensities_list = []
intf = open(intensities_file, 'r')
lines = intf.readlines()
for line in lines:
    data = line.split()
    intensities_list.append(float(data[0]))
intensities_list
intf.close()

In [63]:
# Create Mikko file with real intensities (not dummy)
print(len(generate_intensities))

outputIntsFileStr = path_67p+'67P_LC_NU.dat'
if (os.path.exists(outputIntsFileStr)==True):
    print(f'Just checking you definitely want to overwrite {outputIntsFileStr[-13:]}?')
else: 
    mikkoWrite(outputIntsFileStr,mjd_ltc,generate_intensities,sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,0.0)
# Go and type the number output rom this cell into the top of the file you've just created
# This is the number of lightcurves in total

678
Just checking you definitely want to overwrite 67P_LC_NU.dat?


In [59]:
# Also making standard lightcurve output with dates, magnitudes and 0.01 mag. uncertainty
outputMagsFileStr = path_67p+'67p_20230109_I11_R_no_units.dat'
if (os.path.exists(outputMagsFileStr)==True):
    print(f'Just checking you definitely want to overwrite {outputMagsFileStr[-31:]}?')
else:
    fout = open(path_67p+'67p_20230109_I11_R_no_units.dat', 'w+')
    print('GAME OVER')
    for i,day in enumerate(mjd_ltc):
        magnitude = shifted_mags_list[i]
        fout.write('%.6f %.4f 0.01 %.8f %.8f %.8f %.8f %.8f %.8f %.8f %.8f %.4f %.4f\n'%(day+2400000.5, magnitude, sun_x[i], sun_y[i], sun_z[i], rh_list[i], earth_x[i], earth_y[i], earth_z[i], delta[i], ecl_lon[i], ecl_lat[i]))
    fout.close()

Just checking you definitely want to overwrite 67p_20230109_I11_R_no_units.dat?
